<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/LlamaIndex/3-Sate01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index-core llama-index-utils-workflow llama-index-llms-openai llama-parse llama-index-embeddings-openai llama-index-readers-whisper llama-index-tools-yahoo-finance

In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.tools.yahoo_finance import YahooFinanceToolSpec
from llama_index.core.workflow import Context
from llama_index.core.workflow import JsonPickleSerializer, JsonSerializer

In [4]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]

In [5]:
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

llm = OpenAI(model="gpt-4o-mini")

finance_tools = YahooFinanceToolSpec().to_tool_list()
finance_tools.extend([multiply, add])

In [ ]:
type(finance_tools)

In [ ]:
# 리스트의 길이 출력
print("리스트 길이:", len(finance_tools))

# 리스트의 각 항목 출력
print("리스트 항목들:")
for tool in finance_tools:
    print("-", tool)

In [6]:
workflow = AgentWorkflow.from_tools_or_functions(
    finance_tools,
    llm=llm,
    system_prompt="You are an agent that can perform basic mathematical operations using tools."
)

In [7]:
# configure a context to work with our workflow
ctx = Context(workflow)

In [ ]:
type(ctx)

In [8]:
import asyncio

response = await workflow.run(user_msg="Hi, my name is Laurie!", ctx=ctx)
print(response)

Hello Laurie! How can I assist you today?


In [9]:
response2 = await workflow.run(user_msg="What's my name?", ctx=ctx)
print(response2)

Your name is Laurie! How can I help you today?


In [10]:
# Context를 dictionary 객체로 변환
ctx_dict = ctx.to_dict(serializer=JsonSerializer())

# dictionary로부터 새로운 Context 생성
restored_ctx = Context.from_dict(
    workflow, ctx_dict, serializer=JsonSerializer()
)

response3 = await workflow.run(user_msg="What's my name?", ctx=restored_ctx)
print(response3)

Your name is Laurie. How can I assist you further?


In [14]:
# ctx_dict의 길이 출력
print("ctx_dict의 길이:", len(ctx_dict))

# ctx_dict의 전체 내용 출력 (Key-Value 형태로 보기 좋게 출력)
print("ctx_dict의 내용:")
for key, value in ctx_dict.items():
    print(f"{key}: {value}")

ctx_dict의 길이: 10
ctx_dict의 내용:
globals: {'memory': '{"__is_component": true, "value": {"chat_store": {"store": {"chat_history": [{"role": "user", "additional_kwargs": {}, "blocks": [{"block_type": "text", "text": "Hi, my name is Laurie!"}]}, {"role": "assistant", "additional_kwargs": {}, "blocks": [{"block_type": "text", "text": "Hello Laurie! How can I assist you today?"}]}, {"role": "user", "additional_kwargs": {}, "blocks": [{"block_type": "text", "text": "What\'s my name?"}]}, {"role": "assistant", "additional_kwargs": {}, "blocks": [{"block_type": "text", "text": "Your name is Laurie! How can I help you today?"}]}]}, "class_name": "SimpleChatStore"}, "chat_store_key": "chat_history", "token_limit": 96000, "class_name": "ChatMemoryBuffer"}, "qualified_name": "llama_index.core.memory.chat_memory_buffer.ChatMemoryBuffer"}', 'agents': '["Agent"]', 'state': '{}', 'current_agent_name': '"Agent"', 'handoff_output_prompt': '"Agent {to_agent} is now handling the request due to the followin